# Phi-4 Mini ONNX Paralelní Volání Funkcí - Návod

Tento notebook ukazuje, jak používat Phi-4 Mini s ONNX Runtime GenAI pro paralelní volání funkcí. Volání funkcí umožňuje modelu inteligentně využívat externí nástroje a API na základě požadavků uživatele.

## Přehled

V tomto návodu se naučíte:
- Nastavit Phi-4 Mini s ONNX Runtime GenAI
- Definovat schémata funkcí pro rezervaci letů a hotelů
- Používat řízenou generaci s gramatikou Lark pro strukturovaný výstup
- Provádět paralelní volání funkcí pro komplexní scénáře rezervace cestování

## Předpoklady

Před spuštěním tohoto notebooku se ujistěte, že máte:
- Stažený model Phi-4 Mini ONNX
- Nainstalovaný balíček `onnxruntime-genai`
- Základní znalosti konceptů volání funkcí


## Krok 1: Importujte potřebné knihovny

Nejprve importujeme potřebné knihovny pro implementaci volání funkcí.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Krok 2: Nastavení a konfigurace modelu

Nyní nastavíme model Phi-4 Mini ONNX. Nezapomeňte nahradit cestu skutečným adresářem vašeho modelu.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Krok 3: Nastavení parametrů generování

Nastavte parametry generování, abyste mohli ovládat chování výstupu modelu. Tato nastavení zajišťují deterministické a cílené odpovědi pro volání funkcí.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Krok 4: Definujte dostupné funkce

Zde definujeme funkce, které může náš AI asistent volat. V tomto příkladu máme dvě funkce související s cestováním:
1. **booking_flight_tickets**: Pro rezervaci letů mezi letišti
2. **booking_hotels**: Pro rezervaci ubytování v hotelech

Definice funkcí odpovídají formátu schématu volání funkcí OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Krok 5: Pomocné funkce pro generování gramatiky

Tyto pomocné funkce převádějí definice našich funkcí do formátu gramatiky Lark, který využívá ONNX Runtime GenAI pro řízené generování. To zajišťuje, že model generuje platné volání funkcí ve správném formátu JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Krok 6: Otestujte generování gramatiky

Otestujme naše funkce pro generování gramatiky, abychom zjistili, jak převádějí definice nástrojů do správného formátu.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Krok 7: Připravte systémový prompt a generátor

Nyní vytvoříme systémový prompt, který modelu sdělí dostupné nástroje, a nastavíme generátor s parametry pro řízenou generaci.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Krok 8: Inicializace generátoru s řízeným generováním

Nyní vytvoříme generátor s našimi nakonfigurovanými parametry a použijeme gramatiku Lark pro řízené generování.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Krok 9: Otestujte paralelní volání funkcí

Nyní otestujme naše nastavení na složitém scénáři rezervace cestování, který vyžaduje volání více funkcí.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generování volání funkcí

Model nyní bude generovat strukturované volání funkcí na základě požadavku uživatele. Díky řízené generaci bude výstup ve validním formátu JSON, který lze přímo spustit.

**Očekávaný výstup**: Model by měl generovat volání funkcí pro:
1. `booking_flight_tickets` s detaily letu z Pekingu (PEK) do Paříže (CDG)
2. `booking_hotels` s detaily ubytování v Paříži

Spusťte níže uvedenou buňku pro zobrazení živé generace:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Závěr

🎉 **Gratulujeme!** Úspěšně jste implementovali paralelní volání funkcí s Phi-4 Mini pomocí ONNX Runtime GenAI.

### Co jste se naučili:

1. **Nastavení modelu**: Jak nakonfigurovat Phi-4 Mini s ONNX Runtime GenAI
2. **Definice funkcí**: Jak definovat schémata funkcí pro volání AI funkcí
3. **Řízená generace**: Jak použít gramatiku Lark pro strukturovanou generaci výstupu
4. **Paralelní volání funkcí**: Jak zpracovat složité požadavky vyžadující více volání funkcí

### Klíčové výhody:

- ✅ **Strukturovaný výstup**: Řízená generace zajišťuje platné JSON volání funkcí
- ✅ **Paralelní zpracování**: Možnost zpracovat více volání funkcí v jednom požadavku
- ✅ **Vysoký výkon**: ONNX Runtime poskytuje optimalizované inferenční procesy
- ✅ **Flexibilní schéma**: Snadné přidávání nebo úpravy definic funkcí

### Zdroje:

- [Phi-4 Mini Dokumentace](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentace](https://onnxruntime.ai/docs/genai/)
- [Nejlepší postupy pro volání funkcí](https://platform.openai.com/docs/guides/function-calling)



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho původním jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
